У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings("ignore")

In [2]:
raw_df = pd.read_csv('customer_segmentation_train.csv')

In [3]:
train_df, val_df = train_test_split(raw_df, test_size=0.2, random_state=42, stratify=raw_df['Segmentation'])

In [4]:
train_df['Ever_Married'] = raw_df['Ever_Married'].fillna('Yes')
train_df['Graduated'] = raw_df['Graduated'].fillna('Yes')
train_df['Profession'] = raw_df['Profession'].fillna('Unknown')
train_df['Family_Size'] = raw_df['Family_Size'].fillna('2')
train_df['Var_1'] = raw_df['Var_1'].fillna('Cat_6')
train_df['Work_Experience'] = raw_df['Work_Experience'].fillna(1)

In [5]:
target_col_name = 'Segmentation'
y = train_df[target_col_name]
X = train_df.drop(columns=target_col_name)

y_test = train_df[target_col_name]
X_test = train_df.drop(columns=target_col_name)

# Identifying numeric and categorical columns
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

In [6]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder(drop='if_binary',sparse_output=False, handle_unknown='ignore')
encoder.fit(X[categorical_cols].astype(str))

encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
X[encoded_cols] = encoder.transform(X[categorical_cols].astype(str))
X_test[encoded_cols] = encoder.transform(X_test[categorical_cols].astype(str))

In [7]:
X_train = X[numeric_cols+encoded_cols]
X_val = X_test[numeric_cols+encoded_cols]

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [8]:
from imblearn.over_sampling import SMOTE

# Perform random sampling
smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train.select_dtypes(include=np.number), y)

In [9]:
from imblearn.over_sampling import SMOTENC

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
# Get indices of categorical columns
cat_feature_indices = [X.columns.get_loc(col) for col in categorical_columns]
X = X.apply(lambda col: col.astype(str) if col.dtype == 'object' else col)

sm = SMOTENC(random_state=0, categorical_features=cat_feature_indices)
X_train_smotenc, y_train_smotenc = sm.fit_resample(X, y)

In [10]:
from imblearn.combine import SMOTETomek

smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train.select_dtypes(include=np.number), y)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [11]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

model_smote = LogisticRegression(solver='liblinear')
ovr_model_smote = OneVsRestClassifier(model_smote)
ovr_model_smote.fit(X_train_smote, y_train_smote)
ovr_predictions_smote = ovr_model_smote.predict(X_test.select_dtypes(include=np.number))

print(classification_report(y_test, ovr_predictions_smote))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00      1578
           B       0.00      0.00      0.00      1486
           C       0.24      1.00      0.39      1576
           D       0.00      0.00      0.00      1814

    accuracy                           0.24      6454
   macro avg       0.06      0.25      0.10      6454
weighted avg       0.06      0.24      0.10      6454



In [12]:
model_smotenc = LogisticRegression(solver='liblinear')
ovr_model_smotenc = OneVsRestClassifier(model_smotenc)
ovr_model_smotenc.fit(X_train_smotenc[numeric_cols+encoded_cols], y_train_smotenc)
ovr_predictions_smotenc = ovr_model_smotenc.predict(X_test[numeric_cols+encoded_cols])

print(classification_report(y_test, ovr_predictions_smotenc))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00      1578
           B       0.00      0.00      0.00      1486
           C       0.24      1.00      0.39      1576
           D       0.00      0.00      0.00      1814

    accuracy                           0.24      6454
   macro avg       0.06      0.25      0.10      6454
weighted avg       0.06      0.24      0.10      6454



In [13]:
model_smotetomek = LogisticRegression(solver='liblinear')
ovr_model_smotetomek = OneVsRestClassifier(model_smotetomek)
ovr_model_smotetomek.fit(X_train_smotetomek[numeric_cols+encoded_cols], y_train_smotetomek)
ovr_predictions_smotetomek = ovr_model_smotetomek.predict(X_test[numeric_cols+encoded_cols])

print(classification_report(y_test, ovr_predictions_smotetomek))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00      1578
           B       0.00      0.00      0.00      1486
           C       0.34      0.72      0.46      1576
           D       0.45      0.79      0.58      1814

    accuracy                           0.40      6454
   macro avg       0.20      0.38      0.26      6454
weighted avg       0.21      0.40      0.28      6454



##Conclusion

The SMOTE-Tomek balancing method is the best among the three because it significantly improves the recall and F1-score for classes C and D while achieving higher overall accuracy (0.40 vs. 0.24) compared to SMOTE and SMOTENC. The other two methods fail to improve minority class detection, resulting in nearly zero predictive performance for classes A and B.